## 임베딩

이전 예제에서는 길이가 `vocab_size`인 고차원 bag-of-words 벡터를 사용했으며, 저차원 위치 표현 벡터를 희소한 원-핫 표현으로 명시적으로 변환했습니다. 하지만 이 원-핫 표현은 메모리 효율적이지 않을 뿐만 아니라, 각 단어가 서로 독립적으로 처리됩니다. 즉, 원-핫 인코딩된 벡터는 단어 간의 의미적 유사성을 전혀 표현하지 못합니다.

이번 단원에서는 **News AG** 데이터셋을 계속 탐구할 것입니다. 시작하기 위해 데이터를 로드하고 이전 노트북에서 사용한 몇 가지 정의를 가져오겠습니다.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## 임베딩이란 무엇인가?

**임베딩(embedding)**의 개념은 단어를 낮은 차원의 밀집 벡터로 표현하는 것입니다. 이 벡터는 단어의 의미를 어느 정도 반영합니다. 나중에 의미 있는 단어 임베딩을 만드는 방법에 대해 논의하겠지만, 지금은 임베딩을 단어 벡터의 차원을 줄이는 방법으로 생각해 봅시다.

임베딩 레이어는 단어를 입력으로 받아 지정된 `embedding_size` 크기의 출력 벡터를 생성합니다. 어떤 면에서는 `Linear` 레이어와 매우 유사하지만, 원-핫 인코딩된 벡터를 입력으로 받는 대신 단어 번호를 입력으로 받을 수 있습니다.

네트워크의 첫 번째 레이어로 임베딩 레이어를 사용하면, 우리가 사용하는 모델을 **임베딩 백(embedding bag)** 모델로 전환할 수 있습니다. 이 모델에서는 텍스트의 각 단어를 해당 임베딩으로 변환한 다음, `sum`, `average`, `max`와 같은 집계 함수를 사용해 모든 임베딩에 대해 계산을 수행합니다.

![다섯 개의 시퀀스 단어에 대한 임베딩 분류기를 보여주는 이미지.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

우리의 분류기 신경망은 임베딩 레이어로 시작하여, 그 다음 집계 레이어, 그리고 그 위에 선형 분류기로 구성됩니다:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### 가변적인 시퀀스 크기 처리

이 아키텍처의 결과로 인해, 네트워크에 전달할 미니배치를 특정 방식으로 생성해야 합니다. 이전 단원에서 Bag-of-Words를 사용할 때는, 미니배치 내의 모든 BoW 텐서가 텍스트 시퀀스의 실제 길이와 상관없이 `vocab_size`로 동일한 크기를 가졌습니다. 하지만 단어 임베딩으로 전환하면, 각 텍스트 샘플에 포함된 단어 수가 달라지게 됩니다. 이러한 샘플들을 미니배치로 결합할 때는 패딩을 적용해야 합니다.

이 작업은 데이터 소스에 `collate_fn` 함수를 제공하는 동일한 기술을 사용하여 수행할 수 있습니다:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### 임베딩 분류기 학습

적절한 데이터 로더를 정의했으니, 이전 단원에서 정의한 학습 함수를 사용하여 모델을 학습시킬 수 있습니다:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **참고**: 여기서는 시간 절약을 위해 25,000개의 레코드(전체 에포크보다 적음)만 학습하지만, 여러 에포크 동안 학습을 계속하고 학습률 매개변수를 실험하여 더 높은 정확도를 달성할 수 있습니다. 약 90%의 정확도에 도달할 수 있어야 합니다.


### EmbeddingBag 레이어와 가변 길이 시퀀스 표현

이전 아키텍처에서는 미니배치에 맞추기 위해 모든 시퀀스를 동일한 길이로 패딩해야 했습니다. 이는 가변 길이 시퀀스를 표현하는 가장 효율적인 방법은 아닙니다. 다른 접근법으로는 **offset** 벡터를 사용하는 것이 있습니다. 이 벡터는 하나의 큰 벡터에 저장된 모든 시퀀스의 오프셋을 포함합니다.

![오프셋 시퀀스 표현을 보여주는 이미지](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note**: 위 그림에서는 문자 시퀀스를 보여주고 있지만, 우리의 예제에서는 단어 시퀀스를 다룹니다. 그러나 오프셋 벡터로 시퀀스를 표현하는 일반적인 원리는 동일합니다.

오프셋 표현을 사용하기 위해 [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) 레이어를 사용합니다. 이 레이어는 `Embedding`과 유사하지만, 콘텐츠 벡터와 오프셋 벡터를 입력으로 받으며, 평균화 레이어를 포함합니다. 이 평균화는 `mean`, `sum`, 또는 `max`로 설정할 수 있습니다.

다음은 `EmbeddingBag`을 사용하는 수정된 네트워크입니다:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

훈련을 위한 데이터셋을 준비하려면 오프셋 벡터를 준비할 변환 함수를 제공해야 합니다.


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

이전에 나온 모든 예제와 달리, 이제 우리의 네트워크는 서로 다른 크기의 데이터 벡터와 오프셋 벡터라는 두 개의 매개변수를 받습니다. 마찬가지로, 우리의 데이터 로더도 2개 대신 3개의 값을 제공합니다: 텍스트와 오프셋 벡터가 모두 특징으로 제공됩니다. 따라서, 이를 처리하기 위해 우리의 훈련 함수를 약간 조정해야 합니다:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## 시맨틱 임베딩: Word2Vec

이전 예제에서 모델의 임베딩 레이어는 단어를 벡터 표현으로 매핑하는 방법을 학습했지만, 이 표현은 의미론적 의미가 많지 않았습니다. 비슷한 단어나 동의어가 특정 벡터 거리(예: 유클리드 거리) 측면에서 서로 가까운 벡터로 대응되는 벡터 표현을 학습할 수 있다면 좋을 것입니다.

이를 위해서는 특정 방식으로 대규모 텍스트 컬렉션에서 임베딩 모델을 사전 학습해야 합니다. 시맨틱 임베딩을 학습하는 초기 방법 중 하나는 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)이라고 불립니다. 이는 단어의 분산 표현을 생성하기 위해 사용되는 두 가지 주요 아키텍처를 기반으로 합니다:

 - **연속적 Bag-of-Words** (CBoW) — 이 아키텍처에서는 주변 문맥으로부터 단어를 예측하도록 모델을 학습시킵니다. n그램 $(W_{-2},W_{-1},W_0,W_1,W_2)$가 주어졌을 때, 모델의 목표는 $(W_{-2},W_{-1},W_1,W_2)$로부터 $W_0$를 예측하는 것입니다.
 - **연속적 Skip-Gram** — CBoW와 반대입니다. 이 모델은 현재 단어를 예측하기 위해 주변 문맥 단어의 윈도우를 사용합니다.

CBoW는 더 빠르지만, Skip-Gram은 더 느리며 드문 단어를 표현하는 데 더 효과적입니다.

![단어를 벡터로 변환하는 CBoW와 Skip-Gram 알고리즘을 보여주는 이미지.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Google News 데이터셋에서 사전 학습된 Word2Vec 임베딩을 실험하려면 **gensim** 라이브러리를 사용할 수 있습니다. 아래는 'neural'과 가장 유사한 단어를 찾는 예제입니다.

> **Note:** 처음으로 단어 벡터를 생성할 때, 다운로드하는 데 시간이 걸릴 수 있습니다!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


우리는 단어로부터 벡터 임베딩을 계산하여 분류 모델 훈련에 사용할 수 있습니다 (명확성을 위해 벡터의 첫 20개 구성 요소만 표시합니다):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

문장의 임베딩의 훌륭한 점은 벡터 인코딩을 조작하여 의미를 변경할 수 있다는 것입니다. 예를 들어, 우리는 *king*과 *woman*이라는 단어에 최대한 가까우면서 *man*이라는 단어와는 최대한 멀리 떨어진 벡터 표현을 가진 단어를 찾을 수 있습니다:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW와 Skip-Grams는 모두 "예측 기반" 임베딩으로, 로컬 컨텍스트만을 고려합니다. Word2Vec은 글로벌 컨텍스트를 활용하지 않습니다.

**FastText**는 Word2Vec을 기반으로 각 단어와 단어 내에서 발견되는 문자 n-그램에 대한 벡터 표현을 학습합니다. 이 표현 값들은 각 학습 단계에서 하나의 벡터로 평균화됩니다. 이는 사전 학습에 많은 추가 계산을 요구하지만, 단어 임베딩이 서브워드 정보를 인코딩할 수 있도록 합니다.

또 다른 방법인 **GloVe**는 공기행렬(co-occurrence matrix)의 아이디어를 활용하며, 공기행렬을 더 표현력 있고 비선형적인 단어 벡터로 분해하기 위해 신경망 방법을 사용합니다.

gensim은 여러 가지 단어 임베딩 모델을 지원하므로, FastText와 GloVe로 임베딩을 변경하여 예제를 실험해볼 수 있습니다.


## PyTorch에서 사전 학습된 임베딩 사용하기

위의 예제를 수정하여 임베딩 레이어의 행렬을 Word2Vec과 같은 의미적 임베딩으로 미리 채울 수 있습니다. 사전 학습된 임베딩의 어휘와 우리의 텍스트 코퍼스의 어휘가 일치하지 않을 가능성이 높으므로, 누락된 단어에 대한 가중치는 랜덤 값으로 초기화해야 합니다:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


이제 모델을 훈련시켜 봅시다. 모델을 훈련시키는 데 걸리는 시간이 이전 예제보다 훨씬 더 길다는 점에 유의하세요. 이는 더 큰 임베딩 레이어 크기와 훨씬 더 많은 매개변수 때문입니다. 또한, 이러한 이유로 과적합을 피하려면 더 많은 예제에서 모델을 훈련시켜야 할 수도 있습니다.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

우리의 경우, 정확도가 크게 향상되지 않는 것을 볼 수 있는데, 이는 아마도 상당히 다른 어휘 때문일 가능성이 높습니다.  
서로 다른 어휘 문제를 해결하기 위해 다음과 같은 방법 중 하나를 사용할 수 있습니다:  
* 우리의 어휘로 word2vec 모델을 다시 학습시키기  
* 사전 학습된 word2vec 모델의 어휘를 사용하여 데이터셋을 로드하기. 데이터셋을 로드할 때 사용할 어휘는 로드 과정에서 지정할 수 있습니다.  

후자의 접근 방식이 더 쉬워 보이는데, 특히 PyTorch의 `torchtext` 프레임워크가 임베딩에 대한 내장 지원을 포함하고 있기 때문입니다.  
예를 들어, GloVe 기반 어휘를 다음과 같은 방식으로 인스턴스화할 수 있습니다:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


로드된 어휘는 다음과 같은 기본 작업을 제공합니다:
* `vocab.stoi` 사전은 단어를 사전 인덱스로 변환할 수 있도록 해줍니다.
* `vocab.itos`는 반대로 숫자를 단어로 변환합니다.
* `vocab.vectors`는 임베딩 벡터의 배열로, 단어 `s`의 임베딩을 얻으려면 `vocab.vectors[vocab.stoi[s]]`를 사용해야 합니다.

다음은 임베딩을 조작하여 **kind-man+woman = queen**이라는 방정식을 보여주는 예제입니다 (작동하도록 계수를 약간 조정했습니다):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

GloVe 어휘를 사용하여 데이터셋을 인코딩한 후, 해당 임베딩을 사용하여 분류기를 학습시켜야 합니다.


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

위에서 본 바와 같이, 모든 벡터 임베딩은 `vocab.vectors` 매트릭스에 저장됩니다. 간단한 복사를 통해 임베딩 레이어의 가중치에 이러한 가중치를 로드하는 것이 매우 쉽습니다.


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

우리 데이터셋의 일부 단어가 사전 학습된 GloVe 어휘에 없기 때문에 정확도가 크게 증가하지 않는 이유 중 하나입니다. 따라서 이러한 단어들은 사실상 무시됩니다. 이 문제를 해결하기 위해, 우리는 데이터셋에서 자체 임베딩을 학습시킬 수 있습니다.


## 문맥적 임베딩

Word2Vec와 같은 전통적인 사전 학습 임베딩 표현의 주요 한계 중 하나는 단어 의미의 중의성 문제입니다. 사전 학습된 임베딩은 단어의 문맥적 의미를 어느 정도 포착할 수 있지만, 단어의 모든 가능한 의미가 동일한 임베딩에 인코딩됩니다. 이는 'play'와 같은 많은 단어가 사용되는 문맥에 따라 다른 의미를 가지기 때문에, 후속 모델에서 문제를 일으킬 수 있습니다.

예를 들어, 'play'라는 단어는 다음 두 문장에서 매우 다른 의미를 가집니다:
- 나는 극장에서 **연극**을 봤다.
- 존은 친구들과 **놀고** 싶어한다.

위의 사전 학습된 임베딩은 'play'라는 단어의 두 가지 의미를 동일한 임베딩으로 표현합니다. 이러한 한계를 극복하기 위해서는 **언어 모델**을 기반으로 한 임베딩을 구축해야 합니다. 언어 모델은 방대한 텍스트 코퍼스에서 학습되며, 단어들이 다양한 문맥에서 어떻게 조합될 수 있는지를 *이해*합니다. 문맥적 임베딩에 대한 논의는 이 튜토리얼의 범위를 벗어나지만, 다음 단원에서 언어 모델을 다룰 때 다시 논의할 것입니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
